# documentation.ipynb

**Inhalt / Contents**
- [DE] 1. Einleitung und Ueberblick
- [DE] 2. GUI & Leveldesign
- [DE] 3. Spiellogik
- [DE] 4. Zusammenfassung & Ausblick
- [EN] 1. Introduction & Overview
- [EN] 2. UI & Level Design
- [EN] 3. Game Logic
- [EN] 4. Summary & Outlook


# [DE] 1. Einleitung und Ueberblick

**Spielidee**
- 2D-Plattformer auf JavaFX: USB-Stick einsammeln, Folder-Dateien sichern, Gefahren umgehen, Portal erreichen.
- Kernfokus: klare Spiellogik, Eingaben, visuelles Feedback (Aufgabenblatt: GUI & Logik, keine Atmosphaere).

**Architektur (kurz)**
- `Game`: JavaFX-Start, Config/Sound, Level-Setups, global Physikparameter (`gravity/moveSpeed/jumpPower`).
- `ScreenManager`: Window, InputManager, Game Loop (delta), FPS, Screen-Wechsel.
- `GameScreen`: zentrale Laufzeit (Rendering, Physik, Kamera, HUD, Zustandswechsel).
- `Level` + `TutorialLevel`/`SecondLevel`: Listen fuer `Platform` + `Block`, Layout und Objekte je Szene.
- `Blocks`/`Entities`: Spieler (`EntityPlayer`), Gegner (`RobotEnemyBlock`/`Boss`), Gefahren (`LavaBlock`, `GasBarrierBlock`), Sammelobjekte/Perks (`USBStickBlock`, `FolderBlock`, Boosts), Plattformen (statisch/dynamisch).

**Diagramme (textuell)**
```text
LoadingScreen -> MainMenuScreen -> GameScreen (Level 1/2)
        ^            |                 |
        |            v Finish          v GameOver
        |     LevelFinishedScreen -> MainMenuScreen
```
```text
Input (Keyboard) -> InputManager -> GameScreen.update(delta)
  -> Physik (dx/dy, Gravitation, Kollision)
  -> Kamera (marginX/Y, smoothing)
  -> Levelobjekte (Platforms/Blocks) rel. Kamera
  -> HUD (Hearts, Files, Debug)
```


# [DE] 2. GUI & Leveldesign

## 2.1 Szene & Rendering
- `GameScreen.initialize()`: Root leeren, animierter Hintergrund (`UIUtils.drawAnimatedBackground`), Spieler setzen, Levelobjekte laden/zeichnen, HUD/Buttons/Pause-Overlay.
- `update(delta)`: Physik + Input + Kollisionen, Kamera-Dead-Zone (`marginX/marginY`, `cameraSmooth`), HUD-Refresh.
- `Platform.drawPlatform()`: Tiling auf Basis `Material.FLOOR`, an Kamera ausgerichtet.
- `Block.update()`: Sprites relativ zu Kamera versetzen.

## 2.2 HUD & Punkte
- Files/Score: `countFolderBlocks()` + `countActiveFolders()` -> `Files: X/Y` in `filesProgressLbl`.
- Quest/Tooltips: `questLbl`, `tipsLbl` (F1), `flipperHint` nach Flipper-Erhalt.
- Health: `drawHealth()` mit Herz-Icons; Debug-Bar (F2) fuer FPS/Kamera/Flags; Buttons `Zurueck`/`Pause` via `UIUtils.drawButton`.

## 2.3 Bewegte/veraenderte Objekte
- Spieler: `dx/dy`, Gravitation (`Game.gravity`), Sprung (`jumpPower`), Richtungs-Sprite (`Direction`), NoClip/GodMode ueber F3.
- Dynamische Plattformen: `FloatingPlatformBlock` pendelt Start/Ende mit Animation und `deltaX`-Mitnahme; `EntityMovingPlatform` mit Hover/Sinus.
- Gegner/Projektil: `RobotEnemyBlock` patrouilliert, feuert `LaserBlock` bei Sicht; `BossRobotBlock` nur wenn Spieler oberhalb. `LaserBlock` bewegt sich `speed*delta`, Timeout 4s, 1 Schaden.
- Gefahren/Perks: `LavaBlock` tickt 0.25 HP; `GasBarrierBlock` blockiert bis E + Flipper, fade-out; `JumpBoost`/`SpeedBoost` 10s Buff (PauseTransition), `USBStick` schaltet Sammeln frei, `FolderBlock` ist Score nach Freischaltung.

### Beispiel-Code (Collision/Fire)
```java
// GameScreen: Gegner feuert Laser bei Sicht
if (block instanceof RobotEnemyBlock enemy) {
    LaserBlock laser = enemy.tryFire(player);
    if (laser != null) {
        laser.draw(root);
        pendingBlocks.add(laser);
    }
}
```

## 2.4 Spielzustaende
- Start: `Game.start()` -> `LoadingScreen` -> `MainMenuScreen` -> `GameScreen`.
- Pause: `togglePause()` blendet Overlay (Logik aktuell noch aktiv; TODO fuer echten Halt).
- Game Over: Health <= 0 oder Sturz -> `handleGameOver()` -> MainMenu.
- Level-Ende: `FinishBlock.onCollide` -> `LevelFinishedScreen` -> Button "Naechstes Level" setzt `SecondLevel` + neuen `GameScreen`.

## 2.5 Leveldesign & Wechsel
- Level 1: Bodenplattformen + Lava-Graeben (`addLavaBetweenPlatforms`), USBStick (Sammelfreigabe), Boosts, FloatingPlatformBlock, Finish; `onFinish()` setzt Tutorial-Flag.
- Level 2: dichtere Platforms, Lava-Saeulen, 8 FolderBlocks, JumpBoost-Route, FlipperItem + GasBarrier, BossRobot auf Endplattform, Finish; Start-Sound beim ersten Draw.
- Wechselbedingung: Finish-Kollision -> Abschluss-Screen; USB-Stick Pflicht fuer Score.

## 2.6 Prompts & Entscheidungen
- Kamera-Dead-Zone + smoothing fuer ruhiges HUD.
- F1/F2/F3 entprellt (`pollJustPressed`) fuer Tooltips/Debug/NoClip+GodMode.
- Score als Files statt Zahl -> klares Quest-Feedback.
- Schranke per Flipper + FadeTransition fuer sichtbare Interaktion.
- Levelwechsel ueber separaten Screen, um harte Spruenge zu vermeiden.


# [DE] 3. Spiellogik

## 3.1 Ziel & Ablauf
- Ziel: USB-Stick holen -> Files sammeln -> Finish erreichen, ohne alle Herzen zu verlieren.
- Start: Application-Start -> Menu -> GameScreen; Health 5 Herzen.
- Ende: Health 0, Sturz, oder Finish-Kollision (regulaer) -> Zustandswechsel.

## 3.2 Steuerung & Interaktion
- Tastatur: A/D Laufen, SPACE Springen (nur Boden), E Interaktion (GasBarrier), W/S im NoClip, F1/F2/F3 Toggles.
- InputManager: `isPressed` fuer Dauer, `pollJustPressed` fuer Toggles; Mauspos wird gespeichert, aktuell nicht genutzt.

### Beispiel-Code (Input entprellt)
```java
public boolean pollJustPressed(KeyCode key) {
    boolean hit = justPressedKeys.contains(key);
    justPressedKeys.remove(key);
    return hit;
}
```

## 3.3 Ereignisse & Logik
- Kollision: AABB (`Rectangle2D`), Reihenfolge Platforms -> Blocks; `pendingBlocks` verhindert ConcurrentMod.
- Schaden/Health: Player klemmt Health; GodMode ignoriert Schaden; Lava/Gegner/Lasers reduzieren, Stomp killt Gegner.
- Timer/Delta: Bewegungen und Buff-Dauer per `delta`; Boost-Reset via `PauseTransition(10s)`; Laser-Lebenszeit 4s; `levelStarted` fuer Stats.
- Trennung: Logik in GameScreen/Blocks, Darstellung via UIUtils/Sprites, Kamera-Offsets in `update()`.

## 3.4 OO-Struktur (Auszug)
- `Game`: Lifecycle, Config, Level, Physik-Parameter.
- `ScreenManager`: Loop, FPS, Screens, InputManager.
- `GuiScreen`: Vertrag fuer Screens.
- `GameScreen`: Rendering, Physik, Kamera, HUD, Zustandswechsel.
- `Level` + Implementierungen: Objektlisten, Draw, optional `onFinish`.
- `Platform`: gekachelte Flaechen, Kamera-Update.
- `EntityPlayer`: Sprite/NameTag, Richtung, Flags (NoClip/GodMode/CanCollect/Flipper).
- `FloatingPlatformBlock`/`EntityMovingPlatform`: Bewegung + Mitnahme.
- `RobotEnemyBlock`/`BossRobotBlock` + `LaserBlock`: Gegner/Projektil.
- `LavaBlock`, `GasBarrierBlock`, Perks (`JumpBoost`, `SpeedBoost`, `USBStick`, `Folder`, `Flipper`).

## 3.5 Prompts & Technik
- Render-Thread entkoppeln: Loop-Thread + `Platform.runLater`.
- Stomp-Heuristik verhindert Doppel-Treffer.
- Buff-Reset ueber `PauseTransition` fuer framerate-unabhaengig.
- TODO: `if (paused) return;` zu Beginn von `GameScreen.update` fuer echte Pause.


# [DE] 4. Zusammenfassung & Ausblick

- Stabiler Kern: GameScreen + ScreenManager + Level mit delta-Physik und Kamera-Dead-Zone.
- HUD/Debug: Hearts, Files-Progress, F1/F2/F3, Quest-Hinweise decken Aufgabenblatt-Anforderungen ab.
- Zwei Level: Tutorial (Mechaniken) + SecondLevel (Schranke, Boss, dichteres Platforming).
- Verbesserungen: Pause-Flag in Logik, Level.update() nutzen (z.B. Moving Platforms), mehr HUD-Feedback (Timer/Score), Mausaktionen, Checkpoints, persistente Stats, Kamera-Clamps pro Level.


# [EN] 1. Introduction & Overview

**Game idea**
- 2D platformer (JavaFX): grab the USB stick, collect folder files, avoid hazards, reach the portal. Focus on logic, input, and feedback.

**Architecture (short)**
- `Game`: JavaFX bootstrap, config/sound, level setup, global physics params.
- `ScreenManager`: window, InputManager, game loop (delta), FPS, screen routing.
- `GameScreen`: main runtime (rendering, physics, camera, HUD, states).
- `Level` + `TutorialLevel`/`SecondLevel`: hold `Platform` + `Block` lists and layout.
- `Blocks`/`Entities`: player, enemies, hazards, collectibles, perks, moving platforms.

**Flow (text diagrams)**
```text
LoadingScreen -> MainMenuScreen -> GameScreen (Level 1/2)
        ^            |                 |
        |            v Finish          v GameOver
        |     LevelFinishedScreen -> MainMenuScreen
```
```text
Input -> InputManager -> GameScreen.update(delta)
  -> Physics (dx/dy, gravity, collision)
  -> Camera (dead zone + smoothing)
  -> Level objects relative to camera
  -> HUD (hearts, files, debug)
```


# [EN] 2. UI & Level Design

## 2.1 Scene & rendering
- `initialize()`: clear root, animated background, spawn player, load/draw level, HUD/buttons/pause overlay.
- `update(delta)`: physics, input, collisions, camera dead zone, HUD refresh.
- `Platform.drawPlatform()`: tiled floor with camera offset.
- `Block.update()`: relocate sprites by camera.

## 2.2 HUD & score
- Files: `countFolderBlocks` vs `countActiveFolders` -> `Files: X/Y` label.
- Quest/tooltips: `questLbl`, `tipsLbl` (F1), `flipperHint` after key item.
- Health: heart icons; debug bar (F2) for FPS/camera/flags; back/pause buttons.

## 2.3 Moving / changing objects
- Player: dx/dy, gravity, jump, direction sprite, NoClip/GodMode via F3.
- Moving platforms: `FloatingPlatformBlock` ping-pongs with carry-over; `EntityMovingPlatform` hover/sinus.
- Enemies/projectile: `RobotEnemyBlock` patrol + `tryFire`, `BossRobotBlock` fires only if player above; `LaserBlock` 4s lifetime, 1 damage.
- Hazards/perks: lava tick damage; gas barrier until E + Flipper; jump/speed boosts (10s), USB stick unlocks file collection, folders are score after unlock.

### Code sample (fire logic)
```java
LaserBlock laser = enemy.tryFire(player);
if (laser != null) {
    laser.draw(root);
    pendingBlocks.add(laser);
}
```

## 2.4 Game states
- Start: `Game.start()` -> loading -> menu -> GameScreen.
- Pause: overlay toggle (logic currently still running; future TODO to halt update).
- Game over: HP <= 0 or fall -> back to menu.
- Finish: collision -> LevelFinishedScreen -> next level button loads SecondLevel.

## 2.5 Levels & switching
- Level 1: ground gaps + lava fill, USB stick, boosts, floating platform, finish; `onFinish` sets tutorial flag.
- Level 2: denser layout, lava columns, eight folders, jump-boost route, flipper + gas barrier, boss robot, finish; spawn sound once.
- Switch: finish collision; USB stick required for score collection.

## 2.6 Prompts & decisions
- Dead-zone camera for calm HUD; smoothed tracking.
- Debounced F1/F2/F3 toggles.
- Score as files for clear quest feedback.
- Gate + fade-out for barrier; finish via separate screen to avoid hard jumps.


# [EN] 3. Game Logic

## 3.1 Goal & flow
- Goal: unlock collection, gather files, reach finish alive.
- Start: app -> menu -> GameScreen; HP = 5 hearts.
- End: HP 0, fall, or finish collision.

## 3.2 Input & interaction
- Keyboard: A/D move, SPACE jump (grounded), E interact (barrier), W/S in NoClip, F1/F2/F3 toggles.
- InputManager: held vs just-pressed; mouse stored but unused.

## 3.3 Events & logic
- Collision order: platforms then blocks; pending list for lasers.
- Damage: clamped HP, GodMode ignores damage; lava/enemies/lasers reduce, stomp kills enemy.
- Timing: delta-based movement; boosts reset after 10s; lasers 4s; level start time for stats.
- Separation: logic in GameScreen/blocks, rendering via UI utils, camera offsets in updates.

## 3.4 OO snapshot
- `Game`, `ScreenManager`, `GuiScreen`, `GameScreen`.
- `Level` + impl, `Platform`.
- `EntityPlayer`, moving platforms, enemies + projectiles, hazards, perks/collectibles.

## 3.5 Prompts & tech
- Loop thread + `Platform.runLater` for UI safety.
- Stomp heuristic, debounced toggles, `PauseTransition` for buffs, TODO real pause guard.


# [EN] 4. Summary & Outlook

- Stable core: GameScreen + ScreenManager + Level with delta-physics and dead-zone camera.
- HUD/debug meets assignment (hearts, files, F-keys, quest hints).
- Two levels: tutorial vs. advanced with barrier/boss.
- Next: true paused logic, richer level updates, better HUD (timer/score), mouse actions, checkpoints, persistent stats, camera clamps per level.
